# GreenPower RAG System v07 - Hybrid Versioning

## Pipeline complet:
1. Setup Qdrant (local vectoriel)
2. Chargement documents GreenPower via Gradio
3. **Détection automatique : données stables vs temporelles**
4. Chunking + Embeddings (sentence-transformers)
5. Stockage Qdrant avec **ID intelligents** (stable = écrasement, temporal = historique)
6. RAG pipeline : query → retrieval → **scan content for private_** → filter → Mistral
7. Interface Gradio interactive

## Nouveautés v07 - Versioning Hybride:
- ✨ **Données STABLES** : ID = `{filename}_{chunk_index}` → écrase anciennes versions
  - Ex: politiques, procédures, descriptions produits
- ✨ **Données TEMPORELLES** : ID = `{filename}_{timestamp}_{chunk_index}` → garde historique
  - Ex: prix, salaires, stocks, budgets, KPIs
- ✨ **Détection auto** : patterns `prix`, `tarif`, `salaire`, `stock`, `budget` dans filename/contenu
- ✨ **Métadonnées enrichies** : timestamp, version, type de donnée
- 🔒 **Filtrage private_** : toujours actif sur le contenu

## Comment ça marche:
```
Upload "politique_rh.pdf" → STABLE → écrase l'ancienne version
Upload "prix_2025.csv" → TEMPORAL → garde historique avec timestamp
Upload "prix_2025.csv" (2e fois) → Nouvelle version coexiste avec l'ancienne
```

In [1]:
# Cell 1: Installation des dépendances
!pip install -q langchain-mistralai langchain-community langchain-text-splitters
!pip install -q qdrant-client gradio sentence-transformers
!pip install -q pypdf python-docx python-dotenv

In [2]:
# Cell 2: Imports
import os
import re
from pathlib import Path
from datetime import datetime
import gradio as gr
from langchain_mistralai import ChatMistralAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from typing import List, Tuple
import pypdf
import docx
import json
import csv
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✅ Imports OK")

c:\Users\agaragar\Documents\00 CLIENTS\0000 VALLUP\00 MENSAFLOW\00 CNAM\UseCase\Jupyter\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Imports OK


In [3]:
env_path = Path('.env')
if env_path.exists():
    with open(env_path) as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                os.environ[key.strip()] = value.strip()

MISTRAL_API_KEY = os.getenv('MISTRAL_API_KEY')
QDRANT_URL = os.getenv('QDRANT_URL', ':memory:')  # Use :memory: for local or cloud URL
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY', None)

# 🔒 PATTERN POUR DÉTECTER LES DONNÉES PRIVÉES
PRIVATE_PATTERN = re.compile(r'private_\w+', re.IGNORECASE)

# 📅 PATTERNS POUR DÉTECTER LES DONNÉES TEMPORELLES
TEMPORAL_KEYWORDS = [
    'prix', 'price', 'tarif', 'tarification',
    'salaire', 'salary', 'paie', 'remuneration',
    'stock', 'inventory', 'inventaire',
    'budget', 'kpi', 'metric', 'metriques',
    'vente', 'sales', 'ca', 'chiffre',
    'cours', 'cotation', 'taux', 'rate'
]

In [4]:
# Cell 4: Initialisation des clients

# Configuration CHUNKS
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

# Initialize components
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
llm = ChatMistralAI(model='mistral-small-latest', mistral_api_key=MISTRAL_API_KEY, temperature=0.7)

qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
COLLECTION_NAME = "greenpower_docs"

# Text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
)

print("✅ Clients initialisés")
print(f"   - Qdrant: Mode {QDRANT_URL}")
print(f"   - Embeddings: {EMBEDDING_MODEL}")
print(f"   - LLM: Mistral Small")
print(f"   - 🔒 Pattern privé: {PRIVATE_PATTERN.pattern}")
print(f"   - 📅 Mots-clés temporels: {len(TEMPORAL_KEYWORDS)} patterns")

C:\Users\agaragar\AppData\Local\Temp\ipykernel_16328\3300190634.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)


✅ Clients initialisés
   - Qdrant: Mode https://e7c2066d-6b1b-43d1-be10-9e84939013a4.europe-west3-0.gcp.cloud.qdrant.io
   - Embeddings: sentence-transformers/all-MiniLM-L6-v2
   - LLM: Mistral Small
   - 🔒 Pattern privé: private_\w+
   - 📅 Mots-clés temporels: 23 patterns


In [5]:
# Cell 5: Création de la collection Qdrant

# Obtenir la dimension des embeddings en générant un embedding de test
test_embedding = embeddings.embed_query("test")
VECTOR_SIZE = len(test_embedding)

def create_collection_if_not_exists():
    """Crée la collection Qdrant si elle n'existe pas déjà"""
    try:
        collections = qdrant_client.get_collections()
        collection_names = [c.name for c in collections.collections]
        
        if COLLECTION_NAME in collection_names:
            print(f"ℹ️ Collection '{COLLECTION_NAME}' existe déjà")
            return
        
        qdrant_client.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(
                size=VECTOR_SIZE,
                distance=Distance.COSINE
            )
        )
        print(f"✅ Collection '{COLLECTION_NAME}' créée (dimension: {VECTOR_SIZE})")
    except Exception as e:
        print(f"⚠️ Erreur création collection: {e}")
        raise

def reset_collection():
    """🗑️ Supprime et recrée la collection (nettoyage complet)"""
    try:
        qdrant_client.delete_collection(collection_name=COLLECTION_NAME)
        print(f"🗑️ Collection '{COLLECTION_NAME}' supprimée")
    except:
        pass
    create_collection_if_not_exists()

# Créer la collection
create_collection_if_not_exists()

ℹ️ Collection 'greenpower_docs' existe déjà


In [6]:
# Cell 6: Fonctions utilitaires

def extract_text_from_file(file_path: str) -> str:
    """Extrait le texte d'un fichier PDF, DOCX, TXT, JSON ou CSV"""
    file_path = Path(file_path)
    
    try:
        if file_path.suffix.lower() == '.pdf':
            with open(file_path, 'rb') as f:
                pdf_reader = pypdf.PdfReader(f)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text
        
        elif file_path.suffix.lower() in ['.docx', '.doc']:
            doc = docx.Document(file_path)
            return "\n".join([para.text for para in doc.paragraphs])
        
        elif file_path.suffix.lower() == '.txt':
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        
        elif file_path.suffix.lower() == '.json':
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                if isinstance(data, dict):
                    text_parts = []
                    for key, value in data.items():
                        if isinstance(value, (dict, list)):
                            text_parts.append(f"{key}: {json.dumps(value, ensure_ascii=False, indent=2)}")
                        else:
                            text_parts.append(f"{key}: {value}")
                    return "\n".join(text_parts)
                elif isinstance(data, list):
                    return "\n\n".join([json.dumps(item, ensure_ascii=False, indent=2) for item in data])
                else:
                    return str(data)
        
        elif file_path.suffix.lower() == '.csv':
            with open(file_path, 'r', encoding='utf-8') as f:
                csv_reader = csv.DictReader(f)
                rows = []
                for row in csv_reader:
                    row_text = ", ".join([f"{key}: {value}" for key, value in row.items()])
                    rows.append(row_text)
                return "\n".join(rows)
        
        else:
            return f"Format non supporté: {file_path.suffix}"
    
    except Exception as e:
        return f"Erreur lecture fichier {file_path.name}: {str(e)}"

def contains_private_data(text: str) -> Tuple[bool, List[str]]:
    """🔒 Détecte si le texte contient des valeurs privées (private_xxx)"""
    matches = PRIVATE_PATTERN.findall(text)
    return len(matches) > 0, matches

def is_temporal_data(filename: str, text_content: str) -> bool:
    """📅 Détecte si les données sont temporelles (évoluent dans le temps)
    
    Analyse le nom du fichier et le contenu pour détecter des patterns
    indiquant des données qui changent avec le temps (prix, stocks, etc.)
    """
    filename_lower = filename.lower()
    content_lower = text_content.lower()[:2000]  # Analyser les 2000 premiers caractères
    
    # Chercher les mots-clés temporels
    for keyword in TEMPORAL_KEYWORDS:
        if keyword in filename_lower or keyword in content_lower:
            return True
    
    return False

def chunk_and_embed(text: str, source: str, is_temporal: bool = False) -> List[dict]:
    """📦 Découpe le texte en chunks et génère les embeddings avec versioning intelligent
    
    Args:
        text: Le texte à chunker
        source: Nom du fichier source
        is_temporal: Si True, ajoute timestamp (garde historique), sinon ID stable (écrasement)
    """
    # Chunking
    chunks = text_splitter.split_text(text)
    
    # Embeddings
    chunk_embeddings = embeddings.embed_documents(chunks)
    
    # Timestamp pour données temporelles
    timestamp = datetime.now().strftime("%Y-%m-%d_%H%M%S") if is_temporal else None
    
    # Préparer les points pour Qdrant
    points = []
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        # 🎯 ID INTELLIGENT : stable ou temporal
        if is_temporal:
            # Données temporelles : ID unique avec timestamp (garde historique)
            point_id = f"{source}_{timestamp}_{i}"
        else:
            # Données stables : ID fixe (écrase l'ancienne version)
            point_id = f"{source}_{i}"
        
        point = PointStruct(
            id=point_id,  # Plus uuid.uuid4() !
            vector=embedding,
            payload={
                "text": chunk,
                "source": source,
                "chunk_index": i,
                "is_temporal": is_temporal,
                "timestamp": timestamp,
                "indexed_at": datetime.now().isoformat()
            }
        )
        points.append(point)
    
    return points

print("✅ Fonctions utilitaires définies")
print("   - Détection private_ dans le contenu")
print("   - Détection données temporelles (prix, stocks, etc.)")
print("   - Versioning hybride : stable vs temporal")

✅ Fonctions utilitaires définies
   - Détection private_ dans le contenu
   - Détection données temporelles (prix, stocks, etc.)
   - Versioning hybride : stable vs temporal


In [7]:
# Cell 7: Fonction d'upload de documents avec versioning hybride

def upload_documents(files):
    """📤 Traite et stocke les documents avec versioning intelligent"""
    if not files:
        return "❌ Aucun fichier uploadé"
    
    # S'assurer que la collection existe
    create_collection_if_not_exists()
    
    total_chunks = 0
    temporal_chunks = 0
    stable_chunks = 0
    chunks_with_private_content = 0
    results = []
    
    for file in files:
        try:
            # Extraction du texte
            file_path = file.name if hasattr(file, 'name') else file
            filename = Path(file_path).name
            
            text = extract_text_from_file(file_path)
            
            if text.startswith("Erreur") or text.startswith("Format non supporté"):
                results.append(f"⚠️ {filename}: {text}")
                continue
            
            # 📅 DÉTECTION AUTOMATIQUE : données temporelles ou stables ?
            is_temporal = is_temporal_data(filename, text)
            
            # Chunking et embedding avec versioning intelligent
            points = chunk_and_embed(text, filename, is_temporal=is_temporal)
            
            # 🔍 Analyser combien de chunks contiennent des données privées
            private_chunks_in_doc = 0
            for point in points:
                has_private, _ = contains_private_data(point.payload["text"])
                if has_private:
                    private_chunks_in_doc += 1
            
            # Stockage dans Qdrant (écrase ou ajoute selon le type)
            qdrant_client.upsert(
                collection_name=COLLECTION_NAME,
                points=points
            )
            
            total_chunks += len(points)
            chunks_with_private_content += private_chunks_in_doc
            
            if is_temporal:
                temporal_chunks += len(points)
                version_info = "📅 TEMPORAL (garde historique)"
            else:
                stable_chunks += len(points)
                version_info = "📌 STABLE (écrase ancienne version)"
            
            result_line = f"✅ {filename}: {len(points)} chunks - {version_info}"
            
            if private_chunks_in_doc > 0:
                result_line += f" ⚠️ ({private_chunks_in_doc} avec private_)"
            
            results.append(result_line)
            
        except Exception as e:
            results.append(f"❌ {filename}: Erreur - {str(e)}")
    
    summary = f"\n\n📊 **Résumé d'indexation:**\n"
    summary += f"   - Total: {total_chunks} chunks stockés\n"
    summary += f"   - 📌 Données stables: {stable_chunks} chunks (écrasent anciennes versions)\n"
    summary += f"   - 📅 Données temporelles: {temporal_chunks} chunks (historique conservé)\n"
    
    if chunks_with_private_content > 0:
        summary += f"\n⚠️ {chunks_with_private_content} chunks contiennent des valeurs private_* (seront filtrés)"
    
    return "\n".join(results) + summary

print("✅ Fonction upload_documents définie (versioning hybride)")

✅ Fonction upload_documents définie (versioning hybride)


In [8]:
# Cell 8: Fonction RAG avec filtrage des contenus privés

def search_and_answer(question: str, top_k: int = 3, include_history: bool = False):
    """🔍 Recherche dans Qdrant et génère une réponse (filtre les chunks avec private_)
    
    Args:
        question: Question de l'utilisateur
        top_k: Nombre de chunks à retourner
        include_history: Si True, inclut les versions historiques des données temporelles
    """
    if not question or question.strip() == "":
        return "⚠️ Veuillez poser une question"
    
    try:
        # 1. Générer l'embedding de la question
        question_embedding = embeddings.embed_query(question)
        
        # 2. Recherche dans Qdrant avec un top_k plus large pour compenser le filtrage
        search_results = qdrant_client.search(
            collection_name=COLLECTION_NAME,
            query_vector=question_embedding,
            limit=top_k * 5  # Chercher 5x plus pour avoir assez après filtrage
        )
        
        if not search_results:
            return "⚠️ Aucun document trouvé. Uploadez d'abord des documents."
        
        # 3. 🔒 FILTRAGE DES CHUNKS CONTENANT DES DONNÉES PRIVÉES
        filtered_results = []
        private_chunks = []
        private_values_found = set()
        temporal_info = {}
        
        for point in search_results:
            chunk_text = point.payload["text"]
            
            # Vérifier si le chunk contient des valeurs private_
            has_private, private_matches = contains_private_data(chunk_text)
            
            if has_private:
                # 🚫 Ce chunk contient des données privées, on le stocke mais ne l'utilise pas
                private_chunks.append({
                    'source': point.payload.get('source', 'Unknown'),
                    'matches': private_matches
                })
                private_values_found.update(private_matches)
                continue
            
            # ✅ Chunk clean, on l'ajoute
            filtered_results.append(point)
            
            # Tracker les données temporelles
            if point.payload.get('is_temporal'):
                source = point.payload.get('source')
                timestamp = point.payload.get('timestamp')
                if source not in temporal_info:
                    temporal_info[source] = []
                temporal_info[source].append(timestamp)
            
            # Limiter au top_k demandé
            if len(filtered_results) >= top_k:
                break
        
        # 4. Vérifier s'il reste des résultats après filtrage
        if not filtered_results:
            # Tous les résultats contenaient des données privées
            return f"""## 🔒 Donnée confidentielle

Désolé, les informations pertinentes pour cette question contiennent des **données confidentielles** et ne peuvent pas être affichées.

---

🔒 **{len(private_chunks)} chunks** ont été trouvés mais contenaient des valeurs privées :
{', '.join(sorted(private_values_found))}

💡 Ces données sont protégées et ne seront jamais affichées dans les réponses.
"""
        
        # 5. Extraire les chunks pertinents (uniquement ceux sans données privées)
        contexts = []
        sources = []
        for point in filtered_results:
            contexts.append(point.payload["text"])
            source = point.payload.get("source", "Unknown")
            if source not in sources:
                sources.append(source)
        
        context_text = "\n\n---\n\n".join(contexts)
        
        # 6. Construire le prompt
        prompt = f"""Tu es un assistant qui répond aux questions en te basant UNIQUEMENT sur le contexte fourni.

Contexte:
{context_text}

Question: {question}

Instructions:
- Réponds de manière claire et concise
- Base-toi UNIQUEMENT sur le contexte fourni
- Si l'information n'est pas dans le contexte, dis-le clairement
- Cite les sources quand pertinent

Réponse:"""
        
        # 7. Générer la réponse avec Mistral
        response = llm.invoke(prompt)
        answer = response.content
        
        # 8. Formater la réponse avec info sur filtrage
        output = f"""## 💬 Réponse

{answer}

---

📚 **Sources consultées:** {', '.join(sources)}
🔍 **{len(contexts)} chunks** analysés (sans données confidentielles)
"""
        
        # Ajouter info sur données temporelles
        if temporal_info:
            output += "\n📅 **Données temporelles détectées:**\n"
            for source, timestamps in temporal_info.items():
                latest = max(timestamps) if timestamps else "N/A"
                output += f"   - {source}: version du {latest}\n"
        
        # Ajouter un warning si des chunks privés ont été exclus
        if private_chunks:
            output += f"\n🔒 **{len(private_chunks)} chunks** ont été exclus car ils contenaient des valeurs privées :"
            output += f"\n   {', '.join(sorted(private_values_found))}"
        
        return output
    
    except Exception as e:
        return f"❌ Erreur: {str(e)}\n\nDétails technique: {type(e).__name__}"

print("✅ Fonction search_and_answer définie (filtrage private_ + versioning)")

✅ Fonction search_and_answer définie (filtrage private_ + versioning)


In [9]:
# Cell 9: Interface Gradio

with gr.Blocks(title="GreenPower RAG v07 - Hybrid Versioning") as demo:
    gr.Markdown(
        """
        # 🌱 GreenPower RAG System v07 - 📅 Hybrid Versioning + 🔒 Private Filter
        
        **Retrieval-Augmented Generation** avec **versioning intelligent** et **filtrage des données privées**
        
        ### 📋 Workflow:
        1. **Upload** vos documents (PDF, DOCX, TXT, JSON, CSV)
        2. **Détection automatique** : données stables vs temporelles
        3. **Versioning intelligent** :
           - 📌 Données stables → écrasent l'ancienne version
           - 📅 Données temporelles → garde l'historique avec timestamp
        4. **Ask** vos questions
        5. **Filtrage automatique** des chunks contenant `private_xxx`
        
        ### 🎯 Types de données:
        
        **📌 STABLES** (écrasement) :
        - Politiques, procédures, règlements
        - Descriptions produits, services
        - Documentation technique
        - ID format: `{filename}_{chunk_index}`
        
        **📅 TEMPORELLES** (historique) :
        - Prix, tarifs, cotations
        - Salaires, budgets, KPIs
        - Stocks, inventaires
        - Ventes, chiffre d'affaires
        - ID format: `{filename}_{timestamp}_{chunk_index}`
        
        ### 💡 Exemple:
        ```
        Upload 1: "politique_rh.pdf" → 📌 STABLE
        Upload 2: "prix_2025_v1.csv" → 📅 TEMPORAL (version 2025-01-20_143022)
        Upload 3: "prix_2025_v2.csv" → 📅 TEMPORAL (version 2025-01-20_150533)
        Upload 4: "politique_rh.pdf" (modifié) → 📌 STABLE (écrase la v1)
        
        Résultat: 2 versions de prix coexistent, 1 seule version de politique_rh
        ```
        """
    )
    
    with gr.Tab("📤 Upload Documents"):
        gr.Markdown(
            """
            ### Upload vos documents
            
            Formats supportés: **PDF**, **DOCX**, **TXT**, **JSON**, **CSV**
            
            🤖 **Détection automatique du type de données:**
            
            Le système analyse le **nom du fichier** et le **contenu** pour détecter :
            
            **📅 Données temporelles** (mots-clés détectés) :
            - `prix`, `price`, `tarif`, `tarification`
            - `salaire`, `salary`, `paie`, `remuneration`
            - `stock`, `inventory`, `inventaire`
            - `budget`, `kpi`, `metric`, `metriques`
            - `vente`, `sales`, `ca`, `chiffre`
            - `cours`, `cotation`, `taux`, `rate`
            
            Si détecté → **garde l'historique avec timestamp**
            Sinon → **écrase l'ancienne version**
            
            🔒 **Données privées dans le contenu:**
            - Utilisez le pattern `private_xxx` pour marquer des valeurs sensibles
            - Ex: `"Le client private_client_001 a acheté..."`
            - Ces chunks seront automatiquement filtrés lors des recherches
            """
        )
        
        file_input = gr.File(
            file_count="multiple",
            label="📁 Sélectionnez vos documents",
            file_types=[".pdf", ".docx", ".doc", ".txt", ".json", ".csv"]
        )
        
        with gr.Row():
            upload_btn = gr.Button("🚀 Upload et Traiter", variant="primary")
            reset_btn = gr.Button("🗑️ Reset Collection", variant="stop")
        
        upload_output = gr.Textbox(
            label="📊 Résultat",
            lines=12,
            placeholder="Les résultats d'upload apparaîtront ici..."
        )
        
        upload_btn.click(
            upload_documents,
            inputs=file_input,
            outputs=upload_output
        )
        
        def reset_and_notify():
            reset_collection()
            return "✅ Collection réinitialisée ! Tous les documents ont été supprimés."
        
        reset_btn.click(
            reset_and_notify,
            inputs=None,
            outputs=upload_output
        )
    
    with gr.Tab("💬 Ask Questions"):
        gr.Markdown(
            """
            ### Posez vos questions
            
            Le système va:
            1. 🔍 Rechercher les passages pertinents dans Qdrant
            2. 📅 Prioriser les **versions les plus récentes** pour les données temporelles
            3. 🔒 **Scanner chaque chunk** pour détecter des valeurs `private_*`
            4. 🚫 **Filtrer automatiquement** les chunks contenant des données privées
            5. 🧠 Générer une réponse avec Mistral (uniquement données publiques)
            6. 📚 Citer les sources et versions utilisées
            
            ⚠️ Si tous les chunks pertinents contiennent `private_*` → **"Désolé, donnée confidentielle"**
            """
        )
        
        question_input = gr.Textbox(
            label="❓ Votre question",
            placeholder="Ex: Quels sont les prix actuels? Quelle est notre politique RH?",
            lines=3
        )
        
        top_k_slider = gr.Slider(
            minimum=1,
            maximum=10,
            value=3,
            step=1,
            label="🎯 Nombre de chunks à récupérer",
            info="Plus de chunks = plus de contexte (mais plus lent)"
        )
        
        ask_btn = gr.Button("🤔 Obtenir la Réponse", variant="primary")
        
        answer_output = gr.Markdown(
            label="💡 Réponse",
            value="*La réponse apparaîtra ici...*"
        )
        
        ask_btn.click(
            search_and_answer,
            inputs=[question_input, top_k_slider],
            outputs=answer_output
        )
        
        gr.Examples(
            examples=[
                ["Quels sont les prix actuels?", 3],
                ["Quelle est notre politique de congés?", 3],
                ["Montrez-moi l'évolution des stocks", 5],
                ["Quels sont les objectifs 2025?", 3],
            ],
            inputs=[question_input, top_k_slider],
        )
    
    with gr.Tab("ℹ️ Info"):
        gr.Markdown(
            f"""
            ### 🔧 Configuration Technique
            
            - **Vector DB:** Qdrant ({QDRANT_URL})
            - **Embeddings:** {EMBEDDING_MODEL}
            - **LLM:** Mistral Small
            - **Collection:** {COLLECTION_NAME}
            - **Chunk size:** {CHUNK_SIZE} caractères
            - **Overlap:** {CHUNK_OVERLAP} caractères
            - **🔒 Pattern privé:** `{PRIVATE_PATTERN.pattern}`
            - **📅 Mots-clés temporels:** {len(TEMPORAL_KEYWORDS)} patterns
            
            ### 📅 Système de Versioning Hybride
            
            **Problème résolu:**
            - Certaines données changent souvent (prix, stocks) → besoin d'historique
            - D'autres sont stables (politiques, procédures) → pas besoin d'historique
            - Re-upload d'un fichier : faut-il écraser ou garder les 2 versions ?
            
            **Solution v07:**
            
            1️⃣ **Détection automatique du type de données**
            ```python
            # Analyse filename + contenu
            is_temporal = any(keyword in text for keyword in [
                'prix', 'salaire', 'stock', 'budget', 'kpi', 'vente'...
            ])
            ```
            
            2️⃣ **ID intelligent basé sur le type**
            ```python
            # Données STABLES (écrasement)
            ID = "politique_rh.pdf_5"  # Toujours le même ID
            → Re-upload écrase l'ancien chunk
            
            # Données TEMPORELLES (historique)
            ID = "prix_2025.csv_2025-01-20_143022_5"  # ID unique avec timestamp
            → Re-upload crée un nouveau chunk, garde l'ancien
            ```
            
            3️⃣ **Métadonnées enrichies**
            ```json
            {{
                "text": "...",
                "source": "prix_2025.csv",
                "chunk_index": 5,
                "is_temporal": true,
                "timestamp": "2025-01-20_143022",
                "indexed_at": "2025-01-20T14:30:22"
            }}
            ```
            
            **Cas d'usage concrets:**
            
            📌 **Données STABLES (écrasement)**
            ```
            Upload 1: politique_rh.pdf (v1, 2024)
            → Stocké avec ID: politique_rh.pdf_0, politique_rh.pdf_1...
            
            Upload 2: politique_rh.pdf (v2, 2025, politique mise à jour)
            → ÉCRASE les chunks avec mêmes IDs
            → Seule la v2 reste en base
            
            Avantage: Pas de pollution, toujours la dernière version
            ```
            
            📅 **Données TEMPORELLES (historique)**
            ```
            Upload 1: prix_produits.csv (janvier 2025)
            → Stocké: prix_produits.csv_2025-01-20_100000_0...
            
            Upload 2: prix_produits.csv (février 2025, prix augmentés)
            → Stocké: prix_produits.csv_2025-02-15_140000_0...
            → Les 2 versions COEXISTENT
            
            Question: "Quels sont les prix actuels?"
            → Utilise la version février (plus récente)
            
            Question: "Quelle était l'évolution des prix en janvier?"
            → Peut comparer janvier vs février
            
            Avantage: Analyse d'évolution, traçabilité, audit
            ```
            
            ### 🔒 Système de Confidentialité (inchangé depuis v05)
            
            **Filtrage par contenu:**
            - Regex: `/private_\\w+/` (case insensitive)
            - Appliqué APRÈS la recherche vectorielle
            - Chunks avec `private_xxx` → automatiquement rejetés
            - Message transparent si données filtrées
            
            **Patterns détectés:**
            - `private_client_001` ✅
            - `Private_Salary_Data` ✅
            - `PRIVATE_PROJECT_X` ✅
            
            ### 📝 Notes Importantes
            
            - 📊 Les chunks temporels incluent la version/timestamp dans la réponse
            - 🎯 Recherche vectorielle priorise naturellement les versions récentes (plus de contexte)
            - 🗑️ Utilisez "Reset Collection" pour nettoyer complètement la base
            - 💾 Mode `:memory:` ne persiste pas entre redémarrages
            - 🔄 Pour production: utiliser Qdrant cloud/serveur avec persistence
            
            ### ✨ Nouveautés v07
            
            - ✅ Détection automatique données temporelles vs stables
            - ✅ ID intelligents : stables (écrasement) vs temporels (historique)
            - ✅ Timestamp automatique pour données temporelles
            - ✅ Métadonnées enrichies (is_temporal, timestamp, indexed_at)
            - ✅ Info versions dans les réponses
            - ✅ Bouton Reset Collection pour nettoyage
            - ✅ Compatibilité avec filtrage private_ (v05)
            """
        )

# Lancer l'interface
print("\n" + "="*70)
print("🚀 Lancement de l'interface Gradio v07 - Hybrid Versioning...")
print("="*70)

demo.launch(
    server_name="127.0.0.1",
    server_port=7855,
    share=False,
    show_error=True
)


🚀 Lancement de l'interface Gradio v07 - Hybrid Versioning...
* Running on local URL:  http://127.0.0.1:7855
* To create a public link, set `share=True` in `launch()`.
